# Space Situational Awareness Demo


## Installing the Orbit Prediction Pipeline Tools

First we need to install the [orbit prediction package](https://github.com/IBM/spacetech-ssa/tree/master/orbit_prediction) from the [SSA project](https://github.com/ibm/spacetech-ssa) that will allow us to work with satellite orbit data.

In [ ]:
!pip install -e git+https://github.com/IBM/spacetech-ssa.git#egg=orbit_prediction\&subdirectory=orbit_prediction

## Getting TLE Data

[Two-line element set (TLE)](https://en.wikipedia.org/wiki/Two-line_element_set) is a data format that specifies the orbit of an object around the Earth at a particular point in time. The U.S. government provides an API for downloading TLE data and you can sign up for an account [here](https://www.space-track.org/auth/login). You should enter your account credentials in the cell below.

In [ ]:
SPACETRACK_USERNAME=''
SPACETRACK_PASSWORD=''

Next we will use the SpaceTrack API to download orbit data for the International Space Station (ISS) for the past 30 days. First we import the module that will allow us to ETL the SpaceTrack TLE data.

In [ ]:
import orbit_prediction.spacetrack_etl as etl

Next we create a SpaceTrack API client. The SpaceTrack API is heavily rate limited and this client will keep us compliant with the terms of use.

In [ ]:
spacetrack_client = etl.build_space_track_client(SPACETRACK_USERNAME,
                                                 SPACETRACK_PASSWORD)

We then create an instance that knows how to ETL the raw TLE data into a [pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
spacetrack_etl = etl.SpaceTrackETL(spacetrack_client)

Finally we fetch the raw TLE data and do the ETL. Every satellite in orbit has a [NORAD ID](https://en.wikipedia.org/wiki/Satellite_Catalog_Number) associated with it and the ID for the ISS is `25544`.

In [ ]:
iss_orbit_data = spacetrack_etl.build_leo_df(norad_ids=['25544'],
                                             last_n_days=30,
                                             only_latest=None)

Lets take a look at the resulting DataFrame. Each line corresponds to a given observation of the ISS from the ground. Each observation consists of a timestamp the observation was made called the `epoch` and two 3-dimensional real valued vectors, **r** and **v**,

\begin{equation*}
  \mathbf{r} =
  \begin{pmatrix}
    r_{x}\\
    r_{y}\\
    r_{z}\\
  \end{pmatrix}
  \mathbf{v} =
  \begin{pmatrix}
    v_{x}\\
    v_{y}\\
    v_{z}\\
  \end{pmatrix}
\end{equation*}

corresponding to the position and velocity of the object respectively.

In [ ]:
iss_orbit_data

A detailed description of every column is here:

| Field        | Description                                         | Type     |
|------------ |--------------------------------------------------- |-------- |
| aso\_id      | The unique ID for the ASO                           | string   |
| aso\_name    | The name of the ASO                                 | string   |
| epoch        | The timestamp the orbital observation was taken     | datetime |
| r\_x         | The \`x\` component of the position vector \`r\`    | float    |
| r\_y         | The \`y\` component of the position vector \`r\`    | float    |
| r\_z         | The \`z\` component of the position vector \`r\`    | float    |
| v\_x         | The \`x\` component of the velocity vector \`v\`    | float    |
| v\_y         | The \`y\` component of the velocity vector \`v\`    | float    |
| v\_z         | The \`z\` component of the velocity vector \`v\`    | float    |
| object\_type | Whether the ASO is a paylod, rocket body, or debris | string   |


## Building an ML Training Data Set

In this section we use the ISS orbit data from the last section to build a training data set for our machine learning models. We do this by using a physics-based orbital mechanics model to predict where it thinks the ISS will be for the observations that are 3 days in advance of a given row of our dataset. We can then compare where the ISS actually was based on the radar data and where the physics model says the ISS should have been. This value will be the error in the orbital mechanics model.

In [ ]:
import orbit_prediction.build_training_data as training

This function uses a physics model to predict where the ISS will be based on the radar data.

In [ ]:
physics_model_predicted_orbits = training.predict_orbits(iss_orbit_data,
                                                         last_n_days=None,
                                                         n_pred_days=3)

It added the following columns to our DataFrame:

| Field               | Description                                                                   | Type     |
|------------------- |----------------------------------------------------------------------------- |-------- |
| start\_epoch        | The \`epoch\` when the prediction was started                                 | datetime |
| start\_r\_x         | The \`x\` component of the position vector \`r\` where the prediction started | float    |
| start\_r\_y         | The \`y\` component of the position vector \`r\` where the prediction started | float    |
| start\_r\_z         | The \`z\` component of the position vector \`r\` where the prediction started | float    |
| start\_v\_x         | The \`x\` component of the velocity vector \`v\` where the prediction started | float    |
| start\_v\_y         | The \`y\` component of the velocity vector \`v\` where the prediction started | float    |
| start\_v\_z         | The \`z\` component of the velocity vector \`v\` where the prediction started | float    |
| elapsed\_seconds    | The number of seconds between the \`start\_epoch\` and \`epoch\`              | float    |
| physics\_pred\_r\_x | The \`x\` component of the predicted position vector \`r\`                    | float    |
| physics\_pred\_r\_y | The \`y\` component of the predicted position vector \`r\`                    | float    |
| physics\_pred\_r\_z | The \`z\` component of the predicted position vector \`r\`                    | float    |
| physics\_pred\_v\_x | The \`x\` component of the predicted velocity vector \`v\`                    | float    |
| physics\_pred\_v\_y | The \`y\` component of the predicted velocity vector \`v\`                    | float    |
| physics\_pred\_v\_z | The \`z\` component of the predicted velocity vector \`v\`                    | float    |

In the final part of this section, we calculate the error in the physical model.

In [ ]:
physics_model_errors = training.calc_physics_error(physics_model_predicted_orbits)

Which adds the following to our training data:

| Field              | Description                                                        | Type  |
|------------------ |------------------------------------------------------------------ |----- |
| physics\_err\_r\_x | The prediction error in the \`x\` component of the position vector | float |
| physics\_err\_r\_y | The prediction error in the \`y\` component of the position vector | float |
| physics\_err\_r\_z | The prediction error in the \`z\` component of the position vector | float |
| physics\_err\_v\_x | The prediction error in the \`x\` component of the velocity vector | float |
| physics\_err\_v\_y | The prediction error in the \`y\` component of the velocity vector | float |
| physics\_err\_v\_z | The prediction error in the \`z\` component of the velocity vector | float |


## Training Gradient Boosted Regression Tree Models

Now that we have built a training dataset, our job is to build a machine learning model to predict each of the six `physics_err_` columns. Our baseline approach uses [gradient boosted](https://en.wikipedia.org/wiki/Gradient_boosting) [regression trees](https://en.wikipedia.org/wiki/Decision_tree_learning) (GBRTs) via the popular [XGBoost](https://xgboost.ai) package. First we split the training data we constructed in the previous section into a training and test set. We will use 80% of our data for training and reserve the remaining 20% to evaluate how well our model performs.

In [ ]:
import orbit_prediction.ml_model as ml

train_test_data = ml.build_train_test_sets(physics_model_errors, test_size=0.2)

Next we train six different regression models, one for each `physics_err_` column. Take a look [here](https://xgboost.readthedocs.io/en/latest/parameter.html) for all the possible parameters that can be passed to the underlying models.

In [ ]:
gbrt_params = {'tree_method': 'hist'}
physics_error_model = ml.train_models(train_test_data, params=gbrt_params)

Now that we have trained out models lets see how they performed using both the [root-mean-square error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation) and the [coefficient of determination (R<sup>2</sup>)](https://en.wikipedia.org/wiki/Coefficient_of_determination) as evaluation functions.

In [ ]:
physics_error_model.eval_models(train_test_data['X_test'],
                                train_test_data['y_test'])

Now can you improve upon these results by:

-   Using different parameters for the GBRT models?
-   Using a different machine learning technique?
-   Augmenting the training data or performing feature engineering?